# Risk Assesment and Management
- **Erdem Sirel**
- **Gizem Yıldırım**

# Import and process data

In [136]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os


def import_and_prepare_data():
    path = os.path.join(os.getcwd(), "data")
    files = [file for file in os.listdir(path) if file.endswith("csv")]

    data = pd.DataFrame()
    for file in files:
        file_data = pd.read_csv(os.path.join(path,file))
        file_data["Şirket"] = file[: file.find(" ")]
        data = data.append(file_data)
    return data

try:
    data = pd.read_csv("https://raw.githubusercontent.com/ahmetsirel/ETM959_Risk_Project/master/data.csv", sep=';')
except:
    data = import_and_prepare_data()
    data.to_csv("data.csv", sep=';', index=False)
    print("data prepared from raw files")
    
#Interst Rates
interest_rates = pd.DataFrame([["13 09 2019",	16.50],
                                ["25 10 2019",	14.00],
                                ["13 12 2019",	12.00],
                                ["17 01 2020",	11.25],
                                ["20 02 2020",	10.75],
                                ["18 03 2020",	9.75],
                                ["23 04 2020",	8.75],
                                ["22 05 2020",	8.25],
                                ["25 09 2020",	10.25],
                                ["20 11 2020",	15.00]],
                            columns=["date", "rate"])
interest_rates["date"] = pd.to_datetime(interest_rates["date"], dayfirst=True) 
interest_rates["rate"] = interest_rates["rate"] / 100
interest_rates = interest_rates.set_index("date")
#TODO: Is this the right way to convert the annual interest to weekly interest for our case?
interest_rates = (interest_rates / 365) * 7 #convert to weekly interest
interest_rates = interest_rates.sort_index(ascending=False)

# Process Columns
data.rename({"Fark %": "return"}, axis=1, inplace=True)
data["Tarih"] = pd.to_datetime(data["Tarih"], dayfirst=True)
data[["Şimdi", "Açılış", "Yüksek", "Düşük"]] = data[
    ["Şimdi", "Açılış", "Yüksek", "Düşük"]
].applymap(lambda x: round(float(str(x).replace(",", ".")), 3))
data["return"] = data["return"].apply(
    lambda x: round(float(str(x).replace(",", ".").replace("%", "")) / 100, 3)
)


data = data[["Şirket", "Tarih", "Şimdi", "return"]]

# Calculate net return by substracting the weekly inflation rate from weekly return
def get_interst(date_):
    return interest_rates[interest_rates.index <=date_].head(1)["rate"].values[0]
data["inflation"] = data.apply(lambda row: get_interst(row["Tarih"]) ,axis=1)
data["net_return"] = data["return"] - data["inflation"]

data = data.sort_values(["Şirket", "Tarih"])
data.head()

,Şirket,Tarih,Şimdi,return,inflation,net_return
51,AEFES,2019-12-15,22.32,0.013,0.002301,0.010699
50,AEFES,2019-12-22,23.34,0.046,0.002301,0.043699
49,AEFES,2019-12-29,23.08,-0.011,0.002301,-0.013301
48,AEFES,2020-01-05,23.62,0.023,0.002301,0.020699
47,AEFES,2020-01-12,25.58,0.083,0.002301,0.080699


In [137]:
print(pd.__version__)

1.0.5


# Aggregate data
- Calculate mean and variance for weekly return for each company.
- Create covariance matrix for both return and net return

In [138]:
#Mean & Variance for weekly return
data_agg = data.groupby("Şirket").agg({"return": ["mean", "var"]})
data_agg.columns = data_agg.columns.get_level_values(1)
mean_returns = data_agg["mean"]

#Mean & Variance for weekly net return
data_agg_net = data.groupby("Şirket").agg({"net_return": ["mean", "var"]})
data_agg_net.columns = data_agg_net.columns.get_level_values(1)
mean_net_returns = data_agg_net["mean"]

# Cov for weekly return
cov_data = pd.DataFrame()
for company in data["Şirket"].unique():
    current = data[data["Şirket"] == company]["return"].rename(company).reset_index(drop=True)
    cov_data = pd.concat([cov_data, current], axis=1)
cov = cov_data.cov()

# Cov for weekly net return
cov_data_net = pd.DataFrame()
for company in data["Şirket"].unique():
    current_ = data[data["Şirket"] == company]["net_return"].rename(company).reset_index(drop=True)
    cov_data_net = pd.concat([cov_data_net, current_], axis=1)
cov_net = cov_data_net.cov()

"""
writer = pd.ExcelWriter("risk.xlsx", engine="xlsxwriter")
data.to_excel(writer, sheet_name="Raw Data")
data_agg.to_excel(writer, sheet_name="Aggregated Data")
cov.to_excel(writer, sheet_name="Covariance Matrix for return")
cov_net.to_excel(writer, sheet_name="Cov. Matrix for net return")

writer.save()"""

print("Success")


Success


## Calculate 52 week return for each company
$$r_{52w}=(\prod_{week=1}^{52} (r_{week} + 1)) -1 $$

#### Return

In [139]:
def calculate_return(column):
    return np.prod(column.apply(lambda x: x + 1)) - 1

weekly_returns = cov_data.reset_index(drop=True)
rate_of_total_returns = weekly_returns.apply(calculate_return)
rate_of_total_returns

AEFES   -0.020448
AKBNK   -0.214452
DEVA     1.795690
DGKLB    0.531219
ECILC    0.916592
ENJSA    0.447360
ENKAI    0.186614
GOLTS    0.913725
IHLGM    0.459129
SISE     0.440644
THYAO   -0.103348
TTKOM    0.105694
TTRAK    1.984149
ULKER    0.010131
VESTL    0.711300
dtype: float64

#### Net Return

In [140]:
def calculate_return(column):
    return np.prod(column.apply(lambda x: x + 1)) - 1

weekly_returns_net = cov_data_net.reset_index(drop=True)
rate_of_total_returns_net = weekly_returns_net.apply(calculate_return)
rate_of_total_returns_net

AEFES   -0.113492
AKBNK   -0.289360
DEVA     1.534777
DGKLB    0.386233
ECILC    0.736484
ENJSA    0.311097
ENKAI    0.074330
GOLTS    0.733974
IHLGM    0.321144
SISE     0.304739
THYAO   -0.188680
TTKOM    0.000933
TTRAK    1.706085
ULKER   -0.085710
VESTL    0.550321
dtype: float64

# Calculating the portfolio weights
5.Form twoportfolios using different weights,   
* one where weights increase proportional to decreasing standard deviations of individual assets and 
<blockquote>In order to reverse the variance we used the formula For example this formula will swap the highest and lowest mean in order to reverse the order. </blockquote>
\begin{equation*} \ (var\_for\_p2) \   VAR\_reversed  = MAX(VAR) + MIN(VAR) - VAR \end{equation*} 
* one where weights decrease as average returns decrease.  Calculate the means and variancesof both portfolios.
<blockquote>For this one we had a negative mean return for AKBNK. We took its mean as zero in order to calculate proper weight for portfolio. ("mean_for_p1") </blockquote>


In [141]:
data_agg

,mean,var
Şirket,,
AEFES,0.001135,0.002998
AKBNK,-0.002769,0.003918
DEVA,0.024019,0.009104
DGKLB,0.012962,0.008816
ECILC,0.016327,0.009053
ENJSA,0.008019,0.001804
ENKAI,0.004135,0.001673
GOLTS,0.015673,0.006353
IHLGM,0.011673,0.008453


In [142]:
data_agg["mean_for_p1"] = data_agg["mean"].apply(lambda x: max([x, 0]))
data_agg["var_for_p2"] =  data_agg["var"].sum() / data_agg["var"]

In [167]:
data_agg['Portfolio1'] = data_agg["mean_for_p1"] / data_agg["mean_for_p1"].sum()
data_agg['Portfolio2'] =  data_agg["var_for_p2"] / data_agg["var_for_p2"].sum()

p1_weights = data_agg['Portfolio1']
p2_weights = data_agg['Portfolio2']

print("Check weights", p1_weights.sum(), p2_weights.sum())

Check weights 1.0000000000000002 1.0


In [144]:
data_agg.sort_values("var")

,mean,var,mean_for_p1,var_for_p2,Portfolio1,Portfolio2,Portfolio3
Şirket,,,,,,,
ENKAI,0.004135,0.001673,0.004135,44.889828,0.028667,0.149364,0.022277
ENJSA,0.008019,0.001804,0.008019,41.631775,0.055600,0.138524,0.024020
SISE,0.008385,0.002660,0.008385,28.238994,0.058133,0.093961,0.035412
ULKER,0.001673,0.002933,0.001673,25.609604,0.011600,0.085212,0.039048
AEFES,0.001135,0.002998,0.001135,25.056106,0.007867,0.083371,0.039910
TTKOM,0.003731,0.003567,0.003731,21.060741,0.025867,0.070077,0.047482
AKBNK,-0.002769,0.003918,0.000000,19.174265,0.000000,0.063800,0.052153
THYAO,0.000115,0.004382,0.000115,17.143652,0.000800,0.057043,0.058331
VESTL,0.012788,0.004671,0.012788,16.082334,0.088667,0.053512,0.062180


In [169]:
data_agg.loc["DEVA","var"] / data_agg.loc["ULKER","var"]

3.103871171517335

In [170]:
data_agg.loc["ULKER","Portfolio2"] / data_agg.loc["DEVA","Portfolio2"]

3.103871171517335

# Calculate Portfolio Parameters
 - Mean return of the portfolio (w)
 - Variance of the portfolio
 - Rate of Return of the portfolio (52w)

### Return

#### Portfolio 1

In [145]:
# multiply weights
def mul_weight(row_column):
    
    return np.multiply(row_column, p1_weights)

cov_p1 = cov.apply(mul_weight, axis=1).apply(mul_weight, axis=0)

In [172]:
p1_parameters = pd.DataFrame.from_dict({"Mean return of the portfolio (w)": [np.matmul(p1_weights, mean_returns)],
                                        "Variance of the portfolio": [cov_p1.sum().sum()]}).rename({0:"Portfolio 1"})
p1_parameters

,Mean return of the portfolio (w),Variance of the portfolio
Portfolio 1,0.015832,0.002735


#### Portfolio 2

In [147]:
# multiply weights
def mul_weight2(row_column):
    
    return np.multiply(row_column, p2_weights)

cov_p2 = cov.apply(mul_weight2, axis=1).apply(mul_weight2, axis=0)

In [173]:

p2_parameters = pd.DataFrame.from_dict({"Mean return of the portfolio (w)": [np.matmul(p2_weights, mean_returns)],
                                        "Variance of the portfolio": [cov_p2.sum().sum()]}).rename({0:"Portfolio 2"})
p2_parameters

,Mean return of the portfolio (w),Variance of the portfolio
Portfolio 2,0.007215,0.001573


In [174]:
portfolio_parameters = p1_parameters.append(p2_parameters)
portfolio_parameters

,Mean return of the portfolio (w),Variance of the portfolio
Portfolio 1,0.015832,0.002735
Portfolio 2,0.007215,0.001573


### Net Return

#### Portfolio 1

In [150]:
# multiply weights
def mul_weight(row_column):
    
    return np.multiply(row_column, p1_weights)

cov_p1_net = cov_net.apply(mul_weight, axis=1).apply(mul_weight, axis=0)

In [175]:
p1_parameters_net = pd.DataFrame.from_dict({"Mean net return of the portfolio (w)": [np.matmul(p1_weights, mean_net_returns)],
                                        "Variance of the portfolio": [cov_p1_net.sum().sum()]}).rename({0:"Portfolio 1"})


#### Portfolio 2

In [152]:
# multiply weights
def mul_weight2(row_column):
    
    return np.multiply(row_column, p2_weights)

cov_p2_net = cov.apply(mul_weight2, axis=1).apply(mul_weight2, axis=0)

In [176]:

p2_parameters_net = pd.DataFrame.from_dict({"Mean net return of the portfolio (w)": [np.matmul(p2_weights, mean_net_returns)],
                                        "Variance of the portfolio": [cov_p2_net.sum().sum()]}).rename({0:"Portfolio 2"})


In [177]:
portfolio_parameters_net = p1_parameters_net.append(p2_parameters_net)
portfolio_parameters_net

,Mean net return of the portfolio (w),Variance of the portfolio
Portfolio 1,0.013920,0.002738
Portfolio 2,0.005302,0.001573


# 6.1 Probability of Gaining Positive Return
Assuming the average weekly return of a portfolio is normally distributed with
the parameter values you have collected   
calculate the **probability of gaining a
positive return**   
and the probability of gaining a return above the inflation rate
(rate banks loan money). You need weekly rates. 

In [178]:
portfolio_parameters

,Mean return of the portfolio (w),Variance of the portfolio
Portfolio 1,0.015832,0.002735
Portfolio 2,0.007215,0.001573


In [183]:
portfolio_parameters["z-value"] = (
    (0 - portfolio_parameters["Mean return of the portfolio (w)"]) / 
        portfolio_parameters[ "Variance of the portfolio"] 
                                    )
portfolio_parameters["p-value"] = portfolio_parameters["z-value"].apply(lambda z: stats.norm.sf(abs(z)))
portfolio_parameters["probability"] = 1- portfolio_parameters["p-value"] 

In [184]:
portfolio_parameters

,Mean return of the portfolio (w),Variance of the portfolio,z-value,p-value,probability
Portfolio 1,0.015832,0.002735,-5.788626,3.548233e-09,1.000000
Portfolio 2,0.007215,0.001573,-4.586756,2.250927e-06,0.999998


# 6.1 Probability of Gaining Return Above Inflation
Assuming the average weekly return of a portfolio is normally distributed with
the parameter values you have collected   
calculate the probability of gaining a
positive return   
and the **probability of gaining a return above the inflation rate**
(rate banks loan money). You need weekly rates. 

In [158]:
portfolio_parameters_net

,Rate of Net Return of the portfolio (52w),Mean net return of the portfolio (w),Variance of the portfolio
Portfolio 1,0.842461,0.013920,0.002738
Portfolio 2,0.268249,0.005302,0.001573


In [185]:
portfolio_parameters_net["z-value"] = (
    (0 - portfolio_parameters_net["Mean net return of the portfolio (w)"]) / 
        portfolio_parameters_net[ "Variance of the portfolio"] 
                                    )
portfolio_parameters_net["p-value"] = portfolio_parameters_net["z-value"].apply(lambda z: stats.norm.sf(abs(z)))
portfolio_parameters_net["probability"] = 1- portfolio_parameters_net["p-value"] 

In [186]:
portfolio_parameters_net

,Mean net return of the portfolio (w),Variance of the portfolio,z-value,p-value,probability
Portfolio 1,0.013920,0.002738,-5.083981,1.848023e-07,1.000000
Portfolio 2,0.005302,0.001573,-3.371022,3.744490e-04,0.999626
